In [ ]:
from typing import TypedDict,List,Annotated,Dict
from langgraph.graph import StateGraph,START,END
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools import DuckDuckGoSearchResults #searching tools
from langchain_groq import ChatGroq
from IPython.display import display,Image, Markdown
import os
from datetime import datetime

os.environ["GROQ_API_KEY"] = "your_api_key"
model="llama3-8b-8192"

In [ ]:
llm = ChatGroq(model=model,temperature=0.2)

In [ ]:
class CoachState(TypedDict):
    query:str
    category:str
    response:str

First we are defining utilities we will require further
👉 trim_messages

trim_conversation Function: This function limits the conversation history to the latest messages (up to 10), ensuring only recent and relevant messages are retained in the promp

save_file Function: Saves data into a uniquely timestamped Markdown file in the Agent_output folder, creating the folder if it doesn't exst.

show_md_file Function: Reads and displays the content of a Markdown file within the notebook, rendering it in Markdown form readabilityblity. lity.

In [ ]:
def trim_conversation(prompt):
    """Trims conversation history to retain only the latest messages within the limit."""
    max_messages = 10  # Limit the conversation history to the latest 10 messages
    return trim_messages(
        prompt,
        max_tokens=max_messages,  # Specifies the maximum number of messages allowed
        strategy="last",  # Trimming strategy to keep the last messages
        token_counter=len,  # Counts tokens/messages using the length of the list
        start_on="human",  # Start trimming when reaching the first human message
        include_system=True,  # Include system messages in the trimmed history
        allow_partial=False,  # Ensures only whole messages are included
    )

def save_file(data,filename):
    """Saves data to a markdown file with a timestamped filename."""
    folder_name = "Agent_output" #Folder to store output files
    os.makedirs(folder_name,exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"{filename}_{timestamp}.md"

    file_path = os.path.join(folder_name,filename)

    with open(file_path,"w",encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created succesfully.")
    return file_path

def show_md_file(file_path):
    """Displays the content of a markdown file as Markdown in the notebook."""
    with open(file_path,"r",encoding="utf-8") as file:
        content = file.read()

    display(Markdown(content))

### Now We will create class which will be Responsible for Learning(Tutorial and Q&A sessions)

In [ ]:
class LearningResourcesAgent:
    def __init__(self, prompt):
        self.model = ChatGroq(model = model)
        self.prompt = prompt
        self.tools = [DuckDuckGoSearchResults()]

    def TutorialAgent(self,user_input):
        # Set up an agent with tool access and execute a tutorial-style response
        model_with_tools = self.model.bind_tools(tools=self.tools)
        agent = self.prompt | model_with_tools 
        response = agent.invoke({"input":user_input}).content

        path = save_file(str(response).replace("```markdown","").strip(),"Tutorial")
        print(f"Tutorial saved to {path}")
        return path

    def QueryBot(self, user_input):
        # Initiates a Q&A loop for continuous interaction with the user
        print("\nStarting the Q&A session. Type 'exit' to end the session.\n")
        record_QA_session = []
        record_QA_session.append('User Query: %s \n' % user_input)
        self.prompt.append(HumanMessage(content=user_input))
        while True:
            # Trim conversation history to maintain prompt size
            self.prompt = trim_conversation(self.prompt)
            
            # Generate a response from the AI model and update conversation history
            response = self.model.invoke(self.prompt)
            record_QA_session.append('\nExpert Response: %s \n' % response.content)
            
            self.prompt.append(AIMessage(content=response.content))
            
            # Display the AI's response and prompt for user input
            print('*' * 50 + 'AGENT' + '*' * 50)
            print("\nEXPERT AGENT RESPONSE:", response.content)
            
            print('*' * 50 + 'USER' + '*' * 50)
            user_input = input("\nYOUR QUERY: ")
            record_QA_session.append('\nUser Query: %s \n' % response.content)
            self.prompt.append(HumanMessage(content=user_input))
            
            # Exit the Q&A loop if the user types 'exit'
            if user_input.lower() == "exit":
                print("Ending the chat session.")
                path = save_file(''.join(record_QA_session),'Q&A_Doubt_Session')
                print(f"Q&A Session saved to {path}")
                return path

### Here we are creating Class for Interview handling(Interview Question Prep and MockInterview)


In [ ]:
class InterviewAgent:
    def __init__(self, prompt):
        self.model = ChatGroq(model = model)
        self.prompt = prompt
        self.tools = [DuckDuckGoSearchResults()]

    def Interview_questions(self,user_input):
        # Holds the conversation history and cumulative questions and answers
        chat_history = []
        question_bank = ""
        model_with_tools = self.model.bind_tools(self.tools)
        self.agent = self.prompt | model_with_tools
        while True:
            print("\nStarting the Interview question preparation. Type 'exit' to end the session. \n")
            user_input = input("You")
            if user_input.lower() == "exit":
                print("Ending the conversation. Goodbye!")
                break

            # Generate a response to the user input and add it to questions_bank
            response = self.agent.invoke({"input":user_input,"chat_history":chat_history}).content
            question_bank += str(response).replace("```markdown", "").strip() + "\n"

            chat_history.extend([HumanMessage(content=user_input),response])
            if len(chat_history)>10:
                chat_history = chat_history[-10]

            

        path = save_file(question_bank,"Interview_questions")
        print(f"Interview questions saved to {path}")
        return path


    def Mock_Interview(self):
        # Start a simulated mock interview session
        print("\nStarting the mock interview. Type 'exit' to end the session.\n")
        
        # Initialize with a starting message and store interview records
        initial_message = 'I am ready for the interview.\n'
        interview_record = []
        interview_record.append('Candidate: %s \n' % initial_message)
        self.prompt.append(HumanMessage(content=initial_message))
        
        while True:
            # Trim conversation history if necessary to maintain prompt size
            self.prompt = trim_conversation(self.prompt)
            
            # Generate a response using the chat model
            response = self.model.invoke(self.prompt)
            
            # Add AI response to the conversation history
            self.prompt.append(AIMessage(content=response.content))
            
            # Output the AI's response as the "Interviewer"
            print("\nInterviewer:", response.content)
            interview_record.append('\nInterviewer: %s \n' % response.content)
            
            # Get the user's response as "Candidate" input
            user_input = input("\nCandidate: ")
            interview_record.append('\nCandidate: %s \n' % user_input)
            
            # Add user input to the conversation history
            self.prompt.append(HumanMessage(content=user_input))
            
            # End the interview if the user types "exit"
            if user_input.lower() == "exit":
                print("Ending the interview session.")
                path = save_file(''.join(interview_record),'Mock_Interview')
                print(f"Mock Interview saved to {path}")
                return path

In [ ]:
class ResumeMaker:
    def __init__(self, prompt):
        self.model = ChatGroq(model=model)
        self.prompt = prompt
        self.tools = [DuckDuckGoSearchResults()]

        model_with_tools = self.model.bind_tools(self.tools)
        self.agent = self.prompt | model_with_tools

    def Create_Resume(self, user_input):
        chat_history = []
        # Döngüden sonra kullanılacak son yanıtı saklamak için bir değişken tanımlayalım.
        final_response = None 
        
        print("\nResume making session starting. Type 'exit' if you want to end session.\n")
        
        while True:
            # 1. Çıkış komutunu kontrol et.
            # 'or "quit"' hatası düzeltildi.
            if user_input.lower() in ["exit", "quit"]:
                print("Session ending. Good bye!")
                break
            
            # 2. Agent'ı çalıştır ve yanıtı al.
            # Yanıtı 'response' değil, AIMessage nesnesi olarak alıyoruz.
            response_message = self.agent.invoke({
                "input": user_input,
                "chat_history": chat_history
            })
            
            # Yanıtın içeriğini alıyoruz.
            final_response = response_message.content
            print(f"AI: {final_response}") # Kullanıcıya yanıtı gösterelim.
            
            # 3. Konuşma geçmişini güncelle.
            chat_history.extend([
                HumanMessage(content=user_input), 
                response_message # AIMessage'ın tamamını eklemek daha doğru.
            ])

            # Geçmişi belli bir uzunlukta tut.
            if len(chat_history) > 10:
                chat_history = chat_history[-10:]

            # 4. Kullanıcıdan yeni bir girdi al.
            user_input = input("You: ")

        # Döngü bittiğinde, eğer en az bir yanıt üretilmişse dosyayı kaydet.
        if final_response:
            path = save_file(str(final_response).replace("```markdown", "").strip(), "Resume")
            print(f"CV şu yola kaydedildi: {path}")
            return path
        else:
            # Kullanıcı hemen 'exit' yazarsa hiçbir şey üretilmemiş olur.
            print("No content created for record.")
            return None


In [ ]:
class JobSearch:
    def __init__(self,prompt):
        self.model = ChatGroq(model = model)
        self.prompt = prompt
        self.tools = DuckDuckGoSearchResults()

    def find_jobs(self,user_input):
        results = self.tools.invoke(user_input)
        chain = self.prompt | self.model
        jobs = chain.invoke({"result":results}).content

        path = save_file(str(jobs).replace("```markdown", "").strip(),"Job_search")
        return path

In [ ]:
def categorize(state: CoachState) -> CoachState:
    """Categorizes the user query into one of four main categories: Learn Generative AI Technology, Resume Making, Interview Preparation, or Job Search."""
    
    prompt = ChatPromptTemplate.from_template(
        "Categorize the following customer query into one of these categories:\n"
        "1: Learn Generative AI Technology\n"
        "2: Resume Making\n"
        "3: Interview Preparation\n"
        "4: Job Search\n"
        "Give the number only as an output.\n\n"
        "Examples:\n"
        "1. Query: 'What are the basics of generative AI, and how can I start learning it?' -> 1\n"
        "2. Query: 'Can you help me improve my resume for a tech position?' -> 2\n"
        "3. Query: 'What are some common questions asked in AI interviews?' -> 3\n"
        "4. Query: 'Are there any job openings for AI engineers?' -> 4\n\n"
        "Now, categorize the following customer query:\n"
        "Query: {query}"
    )

    # Creates a categorization chain and invokes it with the user's query to get the category
    chain = prompt | llm 
    print('Categorizing the customer query...')
    category = chain.invoke({"query": state["query"]}).content
    return {"category": category}

def handle_learning_resouce(state:CoachState)->CoachState:
    """Determines if the query is related to Tutorial creation or general Questions on generative AI topics."""
    prompt = ChatPromptTemplate.from_template(
        "Categorize the following user query into one of these categories:\n\n"
        "Categories:\n"
        "- Tutorial: For queries related to creating tutorials, blogs, or documentation on generative AI.\n"
        "- Question: For general queries asking about generative AI topics.\n"
        "- Default to Question if the query doesn't fit either of these categories.\n\n"
        "Examples:\n"
        "1. User query: 'How to create a blog on prompt engineering for generative AI?' -> Category: Tutorial\n"
        "2. User query: 'Can you provide a step-by-step guide on fine-tuning a generative model?' -> Category: Tutorial\n"
        "3. User query: 'Provide me the documentation for Langchain?' -> Category: Tutorial\n"
        "4. User query: 'What are the main applications of generative AI?' -> Category: Question\n"
        "5. User query: 'Is there any generative AI course available?' -> Category: Question\n\n"
        "Now, categorize the following user query:\n"
        "The user query is: {query}\n"
    )  
    chain = prompt | llm
    print("Categorize the customer query futher...")
    response = chain.invoke({"query":state["query"]}).content
    return {"category":response}

def handle_interview_preparation(state:CoachState)->CoachState:
    """Determines if the query is related to Mock Interviews or general Interview Questions."""
    prompt = ChatPromptTemplate.from_template(
        "Categorize the following user query into one of these categories:\n\n"
        "Categories:\n"
        "- Mock: For requests related to mock interviews.\n"
        "- Question: For general queries asking about interview topics or preparation.\n"
        "- Default to Question if the query doesn't fit either of these categories.\n\n"
        "Examples:\n"
        "1. User query: 'Can you conduct a mock interview with me for a Gen AI role?' -> Category: Mock\n"
        "2. User query: 'What topics should I prepare for an AI Engineer interview?' -> Category: Question\n"
        "3. User query: 'I need to practice interview focused on Gen AI.' -> Category: Mock\n"
        "4. User query: 'Can you list important coding topics for AI tech interviews?' -> Category: Question\n\n"
        "Now, categorize the following user query:\n"
        "The user query is: {query}\n"
    )

    chain = prompt |llm
    print("Categorizing the customer query futher...")
    response = chain.invoke({"query":state["query"]}).content
    return{"category":response}
    

### Now we will create function for job search and Resume making


In [ ]:
def job_search(state:CoachState)->CoachState:
    """Provide a job search response based on user query requirements."""
    prompt = ChatPromptTemplate.from_template(
    '''Your task is to refactor and make .md file for the this content which includes
    the jobs available in the market. Refactor such that user can refer easily. Content: {result}'''
    )
    jobSearch = JobSearch(prompt)
    state["query"] = input('Please make sure to mention Job location you want,Job roles\n')
    path = jobSearch.find_jobs(state["query"])
    show_md_file(path)
    return{"response":path}

def handle_resume_making(state:CoachState)->CoachState:
    """Generate a customized resume based on user details for a tech role in AI and Generative AI."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", '''You are a skilled resume expert with extensive experience in crafting resumes tailored for tech roles, especially in AI and Generative AI. 
        Your task is to create a resume template for an AI Engineer specializing in Generative AI, incorporating trending keywords and technologies in the current job market. 
        Feel free to ask users for any necessary details such as skills, experience, or projects to complete the resume. 
        Try to ask details step by step and try to ask all details within 4 to 5 steps.
        Ensure the final resume is in .md format.'''),
       MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    resumeMaker = ResumeMaker(prompt)
    path = resumeMaker.Create_Resume(state["query"])
    show_md_file(path)
    return {"response":path}

### Next we will create a function for Q&A query bot and Tutorial maker

In [ ]:
def ask_query_bot(state:CoachState)->CoachState:
    """Provide detalied answers to user queries related to Generative AI."""
    system_message = '''You are an expert Generative AI Engineer with extensive experience in training and guiding others in AI engineering. 
    You have a strong track record of solving complex problems and addressing various challenges in AI. 
    Your role is to assist users by providing insightful solutions and expert advice on their queries.
    Engage in a back-and-forth chat session to address user queries.'''
    prompt = [SystemMessage(content=system_message)]

    learning_agent = LearningResourcesAgent(prompt)

    path = learning_agent.QueryBot(state["query"])
    show_md_file(path)
    return{"response":path}

def tutorial_agent(state:CoachState)->CoachState:
    """Generate a tutorial blog for Generative AI based on user requirements."""
    system_message = '''You are a knowledgeable assistant specializing as a Senior Generative AI Developer with extensive experience in both development and tutoring. 
         Additionally, you are an experienced blogger who creates tutorials focused on Generative AI.
         Your task is to develop high-quality tutorials blogs in .md file with Coding example based on the user's requirements. 
         Ensure tutorial includes clear explanations, well-structured python code, comments, and fully functional code examples.
         Provide resource reference links at the end of each tutorial for further learning.'''
    prompt = ChatPromptTemplate.from_messages([("system",system_message),
                                              ("placeholder","{chat_history}"),
                                               ("human","{input}"),
                                               ("placeholder","{agent_scratchpad}"),
                                              ])
    learning_agent = LearningResourcesAgent(prompt)
    path = learning_agent.TutorialAgent(state["query"])
    show_md_file(path)
    return {"response":path}
    
    

### Finally we will create function Interview Question prep and Mock interview

In [ ]:
def interview_topics_questions(state:CoachState)->CoachState:
    """Provide a curated list of interview questions related to Generative AI based on user input."""
    system_message = '''You are a good researcher in finding interview questions for Generative AI topics and jobs.
                     Your task is to provide a list of interview questions for Generative AI topics and job based on user requirements.
                     Provide top questions with references and links if possible. You may ask for clarification if needed.
                     Generate a .md document containing the questions.'''
    prompt = ChatPromptTemplate.from_messages([
        ("system",system_message),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}"),
    ])
    interview_agent = InterviewAgent(prompt)
    path = interview_agent.Interview_questions(state["query"])
    show_md_file(path)
    return{"response":path}

def mock_interview(state:CoachState)->CoachState:
    """Conduct a mock interview for a Generative AI position, including evaluation at the end."""
    system_message = '''You are a Generative AI Interviewer. You have conducted numerous interviews for Generative AI roles.
         Your task is to conduct a mock interview for a Generative AI position, engaging in a back-and-forth interview session.
         The conversation should not exceed more than 15 to 20 minutes.
         At the end of the interview, provide an evaluation for the candidate.'''
    prompt = [SystemMessage(content = system_message)]
    interview_agent = InterviewAgent(prompt)
    path = interview_agent.Mock_Interview()
    show_md_file(path)
    return {"response":path}

### Here, We are creating routing function which will be responsible for conditional edge to give direction after categorization.

In [ ]:
def route_query(state:CoachState):
    """Route the query based on its category to the appropriate handler."""
    if '1' in state["category"]:
        print('Category: handle_learning_resource')
        return "handle_learning_resource"  # Directs queries about learning generative AI to the learning resource handler
    elif '2' in state["category"]:
        print('Category: handle_resume_making')
        return "handle_resume_making"  # Directs queries about resume making to the resume handler
    elif '3' in state["category"]:
        print('Category: handle_interview_preparation')
        return "handle_interview_preparation"  # Directs queries about interview preparation to the interview handler
    elif '4' in state["category"]:
        print('Category: job_search')
        return "job_search"  # Directs job search queries to the job search handler
    else:
        print("Please ask your question based on my description.")
        return False  # Returns False if the category does not match any predefined options

def route_interview(state:CoachState)->str:
    """Route the query to the appropriate interview-related handler."""
    if 'Question'.lower() in state["category"].lower():
        print('Category: interview_topics_questions')
        return "interview_topics_questions"  # Directs to the handler for interview topic questions
    elif 'Mock'.lower() in state["category"].lower():
        print('Category: mock_interview')
        return "mock_interview"  # Directs to the mock interview handler
    else:
        print('Category: mock_interview')
        return "mock_interview"  # Defaults to mock interview if category does not clearly match


def route_learning(state: CoachState):
    """Route the query based on the learning path category."""
    if 'Question'.lower() in state["category"].lower():
        print('Category: ask_query_bot')
        return "ask_query_bot"  # Directs queries to the general question bot
    elif 'Tutorial'.lower() in state["category"].lower():
        print('Category: tutorial_agent')
        return "tutorial_agent"  # Directs queries to the tutorial creation agent
    else:
        print("Please ask your question based on my interview description.")
        return False  # Returns False if no clear category match is found

### Now all set lets create workflow graphs adding edges and nodes.

In [ ]:
workflow = StateGraph(CoachState)

workflow.add_node("categorize",categorize)
workflow.add_node("handle_learning_resource",handle_learning_resouce)
workflow.add_node("handle_resume_making",handle_resume_making)
workflow.add_node("handle_interview_preparation",handle_interview_preparation)
workflow.add_node("job_search",job_search)
workflow.add_node("mock_interview",mock_interview)
workflow.add_node("interview_topics_questions",interview_topics_questions)
workflow.add_node("tutorial_agent",tutorial_agent)
workflow.add_node("ask_query_bot",ask_query_bot)

workflow.add_edge(START,"categorize")
workflow.add_conditional_edges(
    "categorize",
    route_query,
        {
        "handle_learning_resource": "handle_learning_resource",
        "handle_resume_making": "handle_resume_making",
        "handle_interview_preparation": "handle_interview_preparation",
        "job_search": "job_search"
    }
)
workflow.add_conditional_edges(
    "handle_interview_preparation",
    route_interview,
    {
        "mock_interview": "mock_interview",
        "interview_topics_questions": "interview_topics_questions",
    }
)
workflow.add_conditional_edges(
    "handle_learning_resource",
    route_learning,
    {
        "tutorial_agent": "tutorial_agent",
        "ask_query_bot": "ask_query_bot",
    }
)
workflow.add_edge("handle_resume_making",END)
workflow.add_edge("job_search",END)
workflow.add_edge("interview_topics_questions", END)
workflow.add_edge("mock_interview", END)
workflow.add_edge("ask_query_bot", END)
workflow.add_edge("tutorial_agent", END)

workflow.set_entry_point("categorize")

app = workflow.compile()

In [ ]:
tutorial_agent

In [ ]:
display(
    Image(
        app.get_graph().draw_mermaid_png()
    )
)

In [ ]:
def run_user_query(query:str)->Dict[str,str]:
    """Process a user query through the LangGraph workflow.
    
    Args:
        query (str): The user's query
        
    Returns:
        Dict[str, str]: A dictionary containing the query's category and response
    """

    results = app.invoke({"query":query})
    return{
        "category":results["category"],
        "response":results["response"]
    }

# ---------------------------Testing Different Scenarios------------------------------

In [ ]:
query = "I want to learn Langchain and langgraph.With usage and concept. Also give coding example implementation for both.Create tutorial for this."
result = run_user_query(query)
result

In [ ]:
query = "I am confused between Langgraph and CrewAI when to use what for Agent Creation?"
result = run_user_query(query)
print(result)

In [ ]:
query = "I want to discussion Interview question for Gen AI job roles."
result = run_user_query(query)
print(result)

In [ ]:
query = "I am confused between Langgraph and CrewAI when to use what for Agent Creation?"
result = run_user_query(query)
print(result)

In [ ]:
query = "I need mock interview to practice."
result = run_user_query(query)
result

In [ ]:
query = "Can you help me to modify my resume based on job description"
result = run_user_query(query)
result

In [ ]:
query = "I want to make resume for Gen AI roles job."
result = run_user_query(query)
result

In [ ]:
query = "I want to search jobs."

result = run_user_query(query)
result